In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device ='cpu'
print ("device:[%s]."%(device))

PyTorch version:[1.7.1].
device:[cpu].


In [90]:
wine_df=pd.read_csv('winequalityN.csv')
wine_target=wine_df['quality']
wine_data=wine_df.drop(columns=['quality'])

train_data,test_data,train_target,test_target=train_test_split(wine_data,wine_target,test_size=0.2)
print(len(train_data),len(test_data))
wine_target.unique()


5197 1300


array([6, 5, 7, 8, 4, 3, 9])

In [91]:
from torch.utils.data import Dataset, DataLoader

class WineDataset(Dataset):
	def __init__(self,mode):
		wine=pd.read_csv('winequalityN.csv')
		wine_target=wine_df['quality']
		wine_data=wine_df.drop(columns=['type','quality'])
		train_data,test_data,train_target,test_target=train_test_split(wine_data,wine_target,test_size=0.2)
		if mode=='Train':
			self.X=train_data
			self.y=train_target
		elif mode=='Test':
			self.X=test_data
			self.y=test_target
		else:
			raise Exception


	def __len__(self):
		len_dataset = None
		len_dataset=len(self.X)

		return len_dataset

	def __getitem__(self, idx):
		X=self.X.loc[idx].to_numpy(dtype='float')
		y=self.y.loc[idx]
		return X, y

In [92]:
train_dataset=WineDataset('Train')
print(train_dataset[0])
train_iter=DataLoader(train_dataset,batch_size=1,num_workers=1)

(array([7.000e+00, 2.700e-01, 3.600e-01, 2.070e+01, 4.500e-02, 4.500e+01,
       1.700e+02, 1.001e+00, 3.000e+00, 4.500e-01, 8.800e+00]), 6)


In [93]:
class MultiLayerPerceptronClass(nn.Module):

    def __init__(self,name='mlp',xdim=11,hdim=10,ydim=7):
        super(MultiLayerPerceptronClass,self).__init__()
        self.name = name
        self.xdim = xdim
        self.hdim = hdim
        self.ydim = ydim
        self.lin_1 = nn.Linear(
			self.xdim,
			self.hdim
        )
        self.lin_2 = nn.Linear(
			self.hdim,
			self.ydim
        )
        self.init_param() # initialize parameters
        
    def init_param(self):
        nn.init.kaiming_normal_(self.lin_1.weight)
        nn.init.zeros_(self.lin_1.bias)
        nn.init.kaiming_normal_(self.lin_2.weight)
        nn.init.zeros_(self.lin_2.bias)

    def forward(self,x):
        net = x
        net = self.lin_1(net)
        net = F.relu(net)
        net = self.lin_2(net)
        return net

M = MultiLayerPerceptronClass().to(device)
loss = nn.MSELoss()
optm = optim.Adam(M.parameters(),lr=1e-3)

In [94]:
M.init_param() # initialize parameters
M.train()
EPOCHS,print_every = 10,1
for epoch in range(EPOCHS):
	loss_val_sum = 0
	for batch_in,batch_out in train_iter:
		print(batch_in)
		y_pred = M.forward(batch_in).to(device)
		loss_out = loss(y_pred,batch_out.to(device))
		optm.zero_grad()

		loss_out.backward()

		optm.step()

		loss_val_sum += loss_out
	loss_val_avg = loss_val_sum/len(train_iter)
	print(loss_val_avg)
print ("Done")        

tensor([[7.0000e+00, 2.7000e-01, 3.6000e-01, 2.0700e+01, 4.5000e-02, 4.5000e+01,
         1.7000e+02, 1.0010e+00, 3.0000e+00, 4.5000e-01, 8.8000e+00]],
       dtype=torch.float64)


RuntimeError: expected scalar type Float but found Double